In [4]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk  # Import ttk for Combobox
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
import pickle
import warnings

# Suppress warnings
warnings.filterwarnings('ignore', message='Compiled the loaded model, but the compiled metrics have yet to be built.')

# Load the necessary files
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('columns.pkl', 'rb') as f:
    column_names = pickle.load(f)

# Load the trained model
model = load_model('fyp.h5')
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load the disease precautions data
precautions_df = pd.read_csv(r"C:\Users\bsush\OneDrive\Desktop\BTech\4-1\Project\Disease Prediction\Disease Prediction Final\Disease precaution.csv")
precautions_df.columns = [col.strip().lower() for col in precautions_df.columns]

# Load the dataset containing symptoms
symptoms_data = pd.read_csv(r"C:\Users\bsush\OneDrive\Desktop\BTech\4-1\Project\Disease Prediction\Disease Prediction Final\DiseaseAndSymptoms.csv")

# Extract unique symptoms from the dataset (all symptom columns)
all_symptoms = set()
for col in symptoms_data.columns[1:]:  # Skip the 'Disease' column
    all_symptoms.update(symptoms_data[col].dropna().unique())  # Extract unique symptoms

all_symptoms = sorted(all_symptoms)  # Sort symptoms alphabetically

# Function to predict disease based on selected symptoms
def predict_disease():
    selected_symptoms = []
    for var in symptom_vars:
        symptom = symptom_vars[var].get()
        if symptom != "Select a symptom":
            selected_symptoms.append(symptom)

    if len(selected_symptoms) < 5:
        messagebox.showerror("Error", "Please select at least 5 symptoms.")
        return

    # Preprocess the input
    symptoms = [symptom.strip().lower() for symptom in selected_symptoms]

    input_df = pd.DataFrame(columns=column_names)
    input_df.loc[0] = 0

    # Set the appropriate columns to 1 based on the input symptoms
    for symptom in symptoms:
        for col in input_df.columns:
            if symptom in col:
                input_df.loc[0, col] = 1

    # Standardize the input
    input_data = scaler.transform(input_df)

    # Make a prediction
    prediction = model.predict(input_data)
    predicted_class = np.argmax(prediction, axis=1)[0]
    
    # Decode the prediction to original label
    disease = label_encoder.inverse_transform([predicted_class])[0]

    # Find the precautions for the predicted disease
    precautions = precautions_df[precautions_df['disease'] == disease][['precaution_1', 'precaution_2', 'precaution_3', 'precaution_4']].values.flatten()

    # Prepare the result text
    precautions_text = "\n".join([f"- {precaution}" for precaution in precautions if isinstance(precaution, str)])

    # Display the result with precautions
    result_label.config(text=f"The predicted disease is: {disease}\n\nPrecautions:\n{precautions_text}")

# Set up the main window
root = tk.Tk()
root.title("Disease Prediction")
root.geometry("600x600")

frame = tk.Frame(root)
frame.pack(pady=20)

symptom_vars = {}

for i in range(1, 6): 
    symptom_vars[i] = tk.StringVar(value="Select a symptom")
    symptom_dropdown = ttk.Combobox(frame, textvariable=symptom_vars[i], values=["Select a symptom"] + all_symptoms, state="readonly", height=10)
    symptom_dropdown.grid(row=i, column=0, padx=10, pady=5)

predict_button = tk.Button(root, text="Predict Disease", command=predict_disease)
predict_button.pack(pady=20)

# Label to display the prediction result
result_label = tk.Label(root, text="The predicted disease will appear here.", font=('Helvetica', 12))
result_label.pack(pady=20)

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
